In [26]:
# Editor appearance set up & Load image

# Extend width of Jupyter Notebook Cell to the size of browser
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


# Import packages needed
import gc
import pickle
import platform
from tkinter import Tk
from tkinter.filedialog import askopenfilename, asksaveasfilename

from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from matplotlib.widgets import RectangleSelector, PolygonSelector
import numpy as np
from skimage import io, draw
import mplcursors
import IPython.display as Disp
from ipywidgets import widgets
import cv2
from osgeo import gdal

import general_funcs


# OS related settings
if platform.system() == 'Windows':
    print('Windows')
    # %matplotlib nbagg
    # Sometimes tk/qt will not let cells rerun after an ERROR occurs
#     %matplotlib tk
    %matplotlib qt
elif platform.system() == 'Darwin':
    print('macOS')
    Tk().withdraw()
    %matplotlib osx
elif platform == 'linux' or platform == 'linux2':
    print('Linux')
# This line of "print" must exist right after %matplotlib command, 
# otherwise JN will hang on the first import statement after this.
print('Interactive plot activated')


# Load image and print size & pre-process

# Use skimage to load multi-layer tiff file
# Tk().withdraw()
image_file = askopenfilename(title='Load image file', initialdir='./data/field_image')
img = io.imread(image_file)
print('Image File:', image_file)
print("Image Shape:", img.shape)

h, w, d = img.shape

# Extract layers from the multilayer tiff file and do some adjustments
if d == 5:
    layer_RGB = img[:, :, 0:3]
    layer_IR = img[:, :, 3]
    layer_mask = img[:, :, 4]
    # Change from 16 bit value to 0~1
    layer_RGB = layer_RGB / 65535
    # Set background to black
    layer_RGB[np.where(layer_mask == 0)] = 0
    layer_IR[np.where(layer_mask == 0)] = 0
    # Set image to show
    img2show = layer_RGB
elif d == 4:
    layer_RGB = img[:, :, 0:3]
    layer_mask = img[:, :, 3]
    # Change from 8 bit value to 0~1
    layer_RGB = layer_RGB / 255
    # Set background to black
    layer_RGB[np.where(layer_mask == 0)] = 0
    # Set image to show
    img2show = layer_RGB
elif d == 2:
    layer_IR = img[:, :, 0]
    layer_mask = img[:, :, 1]
    # Set background to black
    layer_IR[np.where(layer_mask == 0)] = 0
    # Set image to show
    img2show = layer_IR


# Play sound when done
general_funcs.done()

macOS
Interactive plot activated
Image File: /Users/lj/Library/Mobile Documents/com~apple~CloudDocs/Developer/RGB-IR_Field_Image_Processing/data/field_image/BRC/SEPERATED_BRC_FLIR_2019_09_11 - 20%.tif
Image Shape: (6966, 2722, 4)

Done


In [27]:
# Interested area pre-selection - Crop out the image for faster process

fig, ax = plt.subplots(figsize=(7, 7))
plt.subplots_adjust(left=0.05, bottom=0.05, right=0.95, top=0.95)
if d == 5 or d == 4:
    myax = ax.imshow(img2show)
elif d == 2:
    mask_not_0_inds = np.where(layer_mask > 0)
    vmin, vmax = general_funcs.cal_vmin_vmax(img2show, layer_mask)
    myax = ax.imshow(img2show, cmap='gist_gray', vmin=vmin, vmax=vmax)
    cbar = fig.colorbar(myax)

def line_select_callback(eclick, erelease):
    x1, y1 = eclick.xdata, eclick.ydata
    x2, y2 = erelease.xdata, erelease.ydata
    
# def toggle_rect_selector(event):
#     if event.key in ['X', 'x'] and toggle_rect_selector.RS.active:
#         toggle_rect_selector.RS.set_active(False)
#         print('Selector deactivated')
#     if event.key in ['C', 'c'] and not toggle_rect_selector.RS.active:
#         toggle_rect_selector.RS.set_active(True)
#         print('Selector activated')

# toggle_rect_selector.RS = RectangleSelector(ax, line_select_callback, drawtype='box', useblit=True, button=[1], minspanx=50, minspany=50,
#                                             rectprops=dict(facecolor='red', edgecolor='black', alpha=0.1, fill=True), 
#                                             spancoords='pixels', interactive=True)

rs = RectangleSelector(ax, line_select_callback, drawtype='box', useblit=True, button=[1], minspanx=50, minspany=50,
                  rectprops=dict(facecolor='red', edgecolor='black', alpha=0.1, fill=True), spancoords='pixels', interactive=True)


# plt.connect('key_press_event', toggle_rect_selector)
plt.show()

In [29]:
# Record up-left & bottom-right corners of area of interest

interested_area = rs.corners
ul_x, ul_y = np.round([interested_area[0][0], interested_area[1][1]])
br_x, br_y = np.round([interested_area[0][2], interested_area[1][2]])

ul_x = ul_x.astype(int)
ul_y = ul_y.astype(int)
br_x = br_x.astype(int)
br_y = br_y.astype(int)

general_funcs.done()


Done


In [30]:
# Set boundary for each plot

def onselect(vert):
    global one_plot_vertices
    one_plot_vertices = np.round(np.array(vert))
    
def load(b):
    info_box.value = 'Failed'
    
def save(b):
    global one_plot_vertices, ax, ps
    if plot_name_box.value == '':
        info_box.value = 'Failed! Please input plot name!'
        return
    elif plot_name_box.value in plot_vertices.keys():
        info_box.value = 'Failed! Plot already EXISTS!'
        return
    elif one_plot_vertices is None:
        info_box.value = 'Failed! Please select vertices!'
        return
    else:
#         plot_names.append(plot_name_box.value)
#         plot_vertices.append(one_plot_vertices)
        plot_vertices[plot_name_box.value] = one_plot_vertices
        plot_notes[plot_name_box.value] = notes_box.value
        info_box.value = 'Success! Plot ' + plot_name_box.value + ' saved!'
        
        polygon = patches.Polygon(one_plot_vertices, True, facecolor = matplotlib.colors.to_rgba('red', 0.1), edgecolor=matplotlib.colors.to_rgba('orange', 0.5))
        ax.add_patch(polygon)
        text_loc = np.mean(one_plot_vertices, 0)
        ax.text(text_loc[0], text_loc[1], plot_name_box.value, ha='center', va='center')
        
        ps.set_active(False)
        ps = PolygonSelector(ax, onselect, useblit=True)
        
def modify_plot(b):
    global one_plot_vertices, ax
    if one_plot_vertices is None:
        info_box.value = 'Failed! Please select vertices!'
        return
    
    elif not plot_name_box.value in plot_vertices.keys():
        info_box.value = 'Failed! Plot ' + plot_name_box.value + ' does not exist!'
    else:
        keys = list(plot_vertices.keys())
        patch_ind = keys.index(plot_name_box.value)
        patch = ax.patches[patch_ind]
        patch.remove()
        polygon = patches.Polygon(one_plot_vertices, True, facecolor = matplotlib.colors.to_rgba('red', 0.1), edgecolor=matplotlib.colors.to_rgba('orange', 0.5))
        ax.add_patch(polygon)
        text_loc = np.mean(one_plot_vertices, 0)
        text_patch = ax.texts[patch_ind]
        text_patch.remove()
        ax.text(text_loc[0], text_loc[1], plot_name_box.value, ha='center', va='center')
        
        
        plot_vertices.pop(plot_name_box.value)
        plot_notes.pop(plot_name_box.value)
        plot_vertices[plot_name_box.value] = one_plot_vertices
        plot_notes[plot_name_box.value] = notes_box.value
        info_box.value = 'Success! Plot ' + plot_name_box.value + ' modified!'
        
        one_plot_vertices = None
        
def remove_plot(b):
    global ax
    if not plot_name_box.value in plot_vertices.keys():
        info_box.value = 'Failed! Plot ' + plot_name_box.value + ' does not exist!'
    else:
        keys = list(plot_vertices.keys())
        patch_ind = keys.index(plot_name_box.value)
        patch = ax.patches[patch_ind]
        patch.remove()
        text_patch = ax.texts[patch_ind]
        text_patch.remove()
        
        plot_vertices.pop(plot_name_box.value)
        plot_notes.pop(plot_name_box.value)
        info_box.value = 'Success! Plot ' + plot_name_box.value + ' removed!'
    
def save_when_enter_pressed(event):
    if event.key == 'enter':
        save(plot_name_box)
    fig.canvas.draw()


plot_vertices = {}
plot_notes = {}
one_plot_vertices = None

# Text widgets
text_layout = widgets.Layout(width='100%')
plot_name_box = widgets.Text(placeholder='Input plot name here', layout=text_layout)
notes_box = widgets.Textarea(placeholder='Put plot notes here', layout=widgets.Layout(height='50px', width='100%'))
info_label = widgets.Label(value='ERROR/Info will be shown below', layout=text_layout)
info_box = widgets.Label(layout=text_layout)
# Button widgets
button_layout = widgets.Layout(width='auto')
button_load = widgets.Button(description="Load plot name From File (unusable)", layout=button_layout)
button_save = widgets.Button(description="Save Plot (Press Enter)", layout=button_layout)
button_modify = widgets.Button(description='Modify Plot', layout=button_layout, button_style='danger')
button_remove = widgets.Button(description='Remove Plot', layout=button_layout, button_style='danger')

# Box widgets
box_layout = widgets.Layout(width='auto')
button_set1 = widgets.HBox(children=[button_load, button_save], layout = box_layout)
button_set2 = widgets.HBox(children=[button_modify, button_remove], layout = box_layout)
all_widgets = widgets.VBox(children=[plot_name_box, notes_box, button_set1, button_set2, info_label, info_box], layout=box_layout)
display(all_widgets)

button_load.on_click(load)
button_save.on_click(save)
button_modify.on_click(modify_plot)
button_remove.on_click(remove_plot)
plot_name_box.on_submit(save)

out = widgets.Output()
with out:
    plt.close()
    fig, ax = plt.subplots(figsize=(7, 7))
    fig.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9)
    if d == 5 or d == 4:
        ax.imshow(img2show[ul_y:br_y, ul_x:br_x])
    elif d == 2:
        myax = ax.imshow(img2show[ul_y:br_y, ul_x:br_x], cmap='gist_gray', vmin=vmin, vmax=vmax)
        cbar = fig.colorbar(myax)
    ps = PolygonSelector(ax, onselect, useblit=True)
    fig.canvas.mpl_connect('key_press_event', save_when_enter_pressed)
    plt.show()

In [31]:
# Save plot info coordinates to file

ds = gdal.Open(image_file)
gt = ds.GetGeoTransform()

plot_vertices_gps = {}
for plot_name in plot_vertices.keys():
    one_plot_vertices = plot_vertices[plot_name]
    one_plot_vertices_gps = []
    for vertex in one_plot_vertices:
        geo_loc = general_funcs.pix2geo(vertex+[ul_x, ul_y], gt)
        one_plot_vertices_gps.append(geo_loc)
    one_plot_vertices_gps = np.array(one_plot_vertices_gps)
    plot_vertices_gps[plot_name] = one_plot_vertices_gps

def ask_to_save():
#     fn = image_file.split('/')[-2] + '_' + image_file.split('/')[-1].split('.')[0]
    fn = image_file.split('/')[-1].split('.')[0]
    file_name = asksaveasfilename(filetypes=[('pickle', '*.pkl')], title='Save plot locations', initialfile=fn+'_plot_loc', initialdir='./data/plot_location')
    if not file_name:
        return
    if not file_name.endswith('.pkl'):
        file_name += '.pkl'
        
    try:
        with open(file_name, 'wb') as f:
            pickle.dump(interested_area, f)
            pickle.dump(plot_vertices_gps, f)
            pickle.dump(plot_notes, f)
        print('GPS coordinates saved to', file_name)
    except Exception as e:
        showerror(type(e).__name__, str(e))


Tk().withdraw()
ask_to_save()

general_funcs.done()

GPS coordinates saved to /Users/lj/Library/Mobile Documents/com~apple~CloudDocs/Developer/RGB-IR_Field_Image_Processing/data/plot_location/SEPERATED_BRC_FLIR_2019_09_11 - 20%_plot_loc.pkl

Done


In [32]:
plot_vertices2 = {}
for plot_name in plot_vertices_gps.keys():
    one_plot_vertices_gps = plot_vertices_gps[plot_name]
    one_plot_vertices = []
    for vertex in one_plot_vertices_gps:
        pix_loc = general_funcs.geo2pix(vertex, gt)
        one_plot_vertices.append(pix_loc)
    one_plot_vertices = np.array(one_plot_vertices)
    one_plot_vertices = np.round(one_plot_vertices)
    one_plot_vertices = one_plot_vertices.astype(int)
    plot_vertices2[plot_name] = (one_plot_vertices)

In [33]:
print(plot_vertices)
for plot_name in plot_vertices.keys():
    one_plot_vertices = plot_vertices[plot_name]
    print(one_plot_vertices + [ul_x, ul_y])
print(plot_vertices2)
print(plot_vertices_gps)
print(gt)


{'p1': array([[1159.,  143.],
       [1625.,  241.],
       [1589., 1073.],
       [ 963.,  983.]]), 'p2': array([[ 963., 1225.],
       [1553., 1341.],
       [1383., 2495.],
       [ 775., 2423.]]), 'p3': array([[1303., 3345.],
       [ 623., 3237.],
       [ 435., 4847.],
       [ 954., 4928.]])}
[[1414. 1193.]
 [1880. 1291.]
 [1844. 2123.]
 [1218. 2033.]]
[[1218. 2275.]
 [1808. 2391.]
 [1638. 3545.]
 [1030. 3473.]]
[[1558. 4395.]
 [ 878. 4287.]
 [ 690. 5897.]
 [1209. 5978.]]
{'p1': array([[1414, 1193],
       [1880, 1291],
       [1844, 2123],
       [1218, 2033]]), 'p2': array([[1218, 2275],
       [1808, 2391],
       [1638, 3545],
       [1030, 3473]]), 'p3': array([[1558, 4395],
       [ 878, 4287],
       [ 690, 5897],
       [1209, 5978]])}
{'p1': array([[ 20.09937543, 574.8450085 ],
       [184.30472743, 540.3125525 ],
       [171.61933543, 247.1390485 ],
       [-48.96553657, 278.8525285 ]]), 'p2': array([[ -48.96553657,  193.5785045 ],
       [ 158.93394343,  152.7033525 ]